In [1]:
## Load packages ----
import numpy as np
import pandas as pd
import sshtunnel
import psycopg2 as pg
import os
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import urllib.request
from pathlib import Path
import glob

/n/home_fasse/kirene/.conda/envs/dorieh/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [11]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

# Provide the file path
file_path = "../data/input/seda/seda_crosswalk_4.1.dta"

# Read the file using Pandas
df_seda = pd.read_stata(file_path)
df_seda.head()

,sedaschname,ncessch,sedasch,year,fips,fips_op,stateabb,sedafips,sedafipsname,sedalea,...,lcity,lstate,lzip,status,latcod,loncod,geoid,geodef,schnam,imputed
0,Sequoyah Sch - Chalkville Campus,1.000020e+10,1.000020e+10,2009,1.0,1,AL,1.0,Alabama,100002,...,PINSON,AL,36126.0,Open,33.701000,-86.603000,101920.0,0.0,SEQUOYAH SCH - CHALKVILLE CAMPUS,0
1,Sequoyah Sch - Chalkville Campus,1.000020e+10,1.000020e+10,2011,1.0,1,AL,1.0,Alabama,100002,...,PINSON,AL,36126.0,Open,33.699128,-86.606673,101920.0,0.0,SEQUOYAH SCH - CHALKVILLE CAMPUS,0
2,Camps,1.000020e+10,1.000020e+10,2012,1.0,1,AL,1.0,Alabama,100002,...,MT MEIGS,AL,36057.0,Open,32.387825,-86.100736,102430.0,0.0,CAMPS,0
3,Det Ctr,1.000020e+10,1.000020e+10,2012,1.0,1,AL,1.0,Alabama,100002,...,MT MEIGS,AL,36057.0,Open,32.387740,-86.100481,102430.0,0.0,DET CTR,0
4,Wallace Sch - Mt Meigs Campus,1.000020e+10,1.000020e+10,2009,1.0,1,AL,1.0,Alabama,100002,...,MOUNT MEIGS,AL,36057.0,Open,32.308400,-86.102500,102430.0,0.0,WALLACE SCH - MT MEIGS CAMPUS,0


In [5]:
df_seda.columns

Index(['sedaschname', 'ncessch', 'sedasch', 'year', 'fips', 'fips_op',
       'stateabb', 'sedafips', 'sedafipsname', 'sedalea', 'sedaleaname',
       'sedacounty', 'sedacountyname', 'sedametro', 'sedametroname', 'sedacz',
       'leaid', 'leanm', 'countyid', 'countyname', 'metroid13', 'metroname13',
       'czid', 'gslo', 'gshi', 'type', 'leatype', 'locale', 'charter',
       'magnet', 'virtual', 'ever_virtual', 'last_virtual', 'lcity', 'lstate',
       'lzip', 'status', 'latcod', 'loncod', 'geoid', 'geodef', 'schnam',
       'imputed'],
      dtype='object')

In [6]:
df_seda.lzip

0         36126.0
1         36126.0
2         36057.0
3         36057.0
4         36057.0
           ...   
819319      782.0
819320      782.0
819321      782.0
819322      782.0
819323      782.0
Name: lzip, Length: 819324, dtype: float64

In [12]:
# Create a new DataFrame to store the column names and row values
df_output = pd.DataFrame(columns=['Column', 'Row Value'])

# Iterate over each column in df_seda
for column in df_seda.columns:
    # Retrieve the value from the first row of the column
    row_value = df_seda[column].iloc[0]
    
    # Append the column name and row value as a new row in df_output
    df_output = df_output.append({'Column': column, 'Row Value': row_value}, ignore_index=True)

# Display the DataFrame
df_output

,Column,Row Value
0,sedaschname,Sequoyah Sch - Chalkville Campus
1,ncessch,10000200277.0
2,sedasch,10000200277.0
3,year,2009
4,fips,1.0
5,fips_op,1
6,stateabb,AL
7,sedafips,1.0
8,sedafipsname,Alabama
9,sedalea,100002


In [15]:

# Get the distinct years and their counts
year_counts = df_seda['year'].value_counts().sort_index()

# Print the distinct years and their counts
for year, count in year_counts.iteritems():
    print(f"Year: {year}, Count: {count}")

Year: 2009, Count: 76529
Year: 2010, Count: 75691
Year: 2011, Count: 75720
Year: 2012, Count: 75625
Year: 2013, Count: 74881
Year: 2014, Count: 68707
Year: 2015, Count: 74235
Year: 2016, Count: 74270
Year: 2017, Count: 75440
Year: 2018, Count: 73590
Year: 2019, Count: 74636
